## Gabriel Akio Urakawa - 11795912
## Lucas Xavier Leite - 10783347

Projeto - Etapa 2  
SCC0252 - Visualização Computacional (2023)
Profa Maria Cristina Ferreira de Oliveira

# Introdução

Para essa etapa do trabalho não usamos o google collab, foi utilizado jupyter e rodado nos localhost de porta 8050 e 8052.

Foi escolhido o database do IMBD TOP1000 fornecido pela professora. Uma vez que os dados foram baixados, realizamos o upload em repositório pessoal do GitHub e realizamos o trabalho. Primeiramente observamos os dados e fizemos um pré-processamento de dados pelo que achamos cabível.

In [10]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import dash
from dash import dcc
from dash import html

In [11]:
# obtencao da base de dados do kaggle que foi colocada no github
import wget
if not os.path.isfile('imdb_top_1000.csv'):
  !python -m wget https://raw.githubusercontent.com/GabrielAkioUrakawa/Visualizacao-Computacional/main/archive/imdb_top_1000.csv

## Conjunto de dados

# Pré-procesamento

Considerando que o pré-processamento é igual à da primeira entrega, tiramos os prints que mostravam parte do raciocínio e encurtamos os comentários dessa primeira parte.

Usando um df.info() observamos que, em primeiro momento, haviam 3 colunas que haviam valores invalidos para nossas visualizações (Certificate, Meta_score e Gross), percebemos também que a receita (Gross), a data de lançamento e o tempo de filme (Runtime) estavam em tipo de dado inadequados. Além disso, os valores da coluna Genre está agrupada de forma inadequada pra visualização e será manipulada posteriormente

Realizando as devidas alterações, verificamos a existência de algum valor repetido ou inválido. No caso de valores inválidos, decidimos criar um novo dataframe chamado dfClean para armazenar os dados limpos. Assim, escolheríamos entre usar df e dfClean de acordo com os dados utilizados em cada visualização. Além disso, consideramos as colunas Certificate, Overview e Poster_Link como irrelevantes para nossa aplicação e, assim, "dropamos", ou removemos, as três colunas.

In [12]:
# preprocessamento de dados
df = pd.read_csv('./imdb_top_1000.csv', encoding='latin-1')

df['Gross'] = df['Gross'].replace(',','', regex=True)
df['Gross'] = pd.to_numeric(df['Gross'], errors='coerce', downcast='integer')
df['Released_Year'] = pd.to_numeric(df['Released_Year'], errors='coerce', downcast='integer')
df['Runtime'] = df['Runtime'].replace(" min",'', regex=True)
df['Runtime'] = pd.to_numeric(df['Runtime'], errors='coerce', downcast='integer')


df = df.drop(['Certificate'], axis=1)
df = df.drop(['Overview'], axis=1)
df = df.drop(['Poster_Link'], axis=1)

dfClean = df.dropna()

# Visualização

### Observações sobre os filmes em relação a ano e gênero

Pensamos em observar como os gêneros dos filmes foram evoluindo ao longo dos anos, assim fizemos duas visualizações nesse dashboard para comparar e chegar em conclusões, por quantidade lançada e por nota máxima. Por exemplo, 

Para a quantidade lançada, fizemos um gráfico de barras agrupadas para visualizar o total somado todos os gêneros e conseguir ver o impacto de cada gênero no total.

Para a nota máxima, fizemos um gráfico de barras agrupada para comparar facilmente entre dois gêneros pelos anos e observar padrões.

Com as nossas visualização podemos observar o crescimento das notas e da quantidade de filmes lançados pelo gênero e intervalo de anos que o usuário desejar, assim, pode-se observar se existe um padrão de lançamento ou não. Também podemos ver o crescimento da industria do cinema em relação a diferentes gêneros e ver a qualidade desses filmes segundo críticos. 

In [27]:
movies_genres = dfClean.drop('Genre', axis=1).join(dfClean.Genre.str.split(", ", expand=True).stack().reset_index(drop=True, level=1).rename('Genre'))

movies_options = list(movies_genres["Genre"].unique())
years_options = list(movies_genres["Released_Year"].sort_values().unique().astype('int32'))

app = dash.Dash()

app.layout = html.Div([
    html.H2("Selecione os gêneros a serem visualizados:", style={"text-align": "center"}),
    html.Div(                        
        [   
            html.Div([
                dcc.Checklist(
                    id="Genres",
                    options=movies_options,
                    value=["Drama"],
                    inline=True,
                    style={
                        "display": 'flex',
                        "padding": "5px 10px",
                        "flex-wrap": "wrap",
                        "justify-content": "space-between",
                        "align-items": "center",
                        "width" :"70%"
                        }
                )], style={"display": 'flex',"justify-content": "center"}
            ),
            html.Div([
                html.H2("Selecione o intervalo de anos a serem visualizados:", style={"text-align": "center"}),
                dcc.RangeSlider(
                    years_options[1],           
                    2025, 
                    10, 
                    marks={i: f'{i}' for i in range(years_options[1], 2026, 10)}, 
                    value=[1985, 2005], 
                    id='Years'
                )]
            )
        ],
        style={'width': '100%',              
               'display': 'inline-block'}),
    dcc.Graph(id='graph1'),
    dcc.Graph(id='graph2'),
], style={"display": 'flex', "flex-direction": "column", 'height':'100%'})


@app.callback(
    dash.dependencies.Output('graph1', 'figure'),
    [dash.dependencies.Input('Genres', 'value'), dash.dependencies.Input('Years', 'value')])
def update_graph1(Genres, Years):
    df_plot = movies_genres[movies_genres['Genre'].isin(Genres)]
    df_plot = df_plot[df_plot["Released_Year"] > Years[0]]
    df_plot = df_plot[df_plot["Released_Year"] < Years[1]]

    pv = pd.pivot_table(
        df_plot,
        index=['Released_Year'],
        columns=["Genre"],
        values=["Runtime"],
        aggfunc='count',
        fill_value=0) 
    
    traces = [go.Bar(x=pv.index, y=pv[('Runtime', g)], name=g) for g in Genres]
    return {
        'data': traces,
        'layout':
        go.Layout(
            title='Quantidade de filmes lançados do(s) gênero(s) {} por ano'.format(Genres),
            xaxis=dict(title='Ano'),
            yaxis=dict(title='Quantidade de Filmes Lançados'),
            barmode='stack'
            )
    }
@app.callback(
    dash.dependencies.Output('graph2', 'figure'),
    [dash.dependencies.Input('Genres', 'value'), dash.dependencies.Input('Years', 'value')])
def update_graph2(Genres, Years):
    movies_genres['Released_Year'] = pd.to_numeric(movies_genres['Released_Year'], errors='raise', downcast='integer')
    df_plot = movies_genres[(movies_genres['Genre'].isin(Genres))]
    df_plot = df_plot[df_plot["Released_Year"] > Years[0]]
    df_plot = df_plot[df_plot["Released_Year"] < Years[1]]

    pv = pd.pivot_table(
        df_plot,
        index=['Released_Year'],
        columns=["Genre"],
        values=["Meta_score"],
        aggfunc='mean',
        fill_value=0) 
    # g4 = sns.catplot(data=selected_genres, x='Released_Year', y='Meta_score', hue='Genre', kind='bar', errorbar=None, height=12, aspect=2, palette='bright')

    traces = [go.Bar(x=pv.index, y=pv[('Meta_score', g)], name=g) for g in Genres]
    layout = go.Layout(
            title='Nota média do(s) gênero(s) {} por ano'.format(Genres),
            xaxis=dict(title='Ano'),
            yaxis=dict(title='Nota média obtida'),
            barmode='group'
            )
    return {
        'data': traces,
        'layout': layout
    }

if __name__ == '__main__':
    app.run_server(debug=True)

### Receita média de filmes por gênero e ano

Finalmente, queremos visualizar a receita média de acordo com o ano de lançamento. Para acomodar a visualização, selecionamos apenas alguns gêneros de filmes. Com as duas informações em mãos, optamos por criar um mapa de calor (heatmap) da receita média de cada gênero escolhido de acordo com o ano de lançamento. Ele complementa a visualização do dashboard anterior e permite a comparação de modo visualmente facilitado. De modo análogo ao anterior, podemos selecionar intervalo de ano e os gêneros a serem visualizados.

In [44]:
movies_genres = dfClean.drop('Genre', axis=1).join(dfClean.Genre.str.split(", ", expand=True)).rename(columns={0: 'Genre 1', 1: 'Genre 2', 2: 'Genre 3'})
movies_genres = movies_genres[['Genre 1','Genre 3','Genre 2', 'Released_Year', 'Gross']]

app2 = dash.Dash()

app2.layout = html.Div([
    html.H2("Selecione os gêneros a serem visualizados:", style={"text-align": "center"}),
    html.Div(                        
        [   
            html.Div(
                dcc.Checklist(
                    id="Genres",
                    options=movies_options,
                    value=['Action', 'Mystery', 'Sci-Fi', 'Thriller', 'Comedy', 'Romance'],
                    inline=True,
                    style={
                        "display": 'flex',
                        "padding": "5px 10px",
                        "flex-wrap": "wrap",
                        "justify-content": "space-between",
                        "align-items": "center",
                        "width" :"70%"
                        }
                ), style={"display": 'flex',"justify-content": "center"}
            ),
            html.Div([
                html.H2("Selecione o intervalo de anos a serem visualizados:", style={"text-align": "center"}),
                dcc.RangeSlider(
                    2000,           
                    2020, 
                    10, 
                    marks={i: f'{i}' for i in range(2000, 2021, 1)}, 
                    value=[2005, 2015], 
                    id='Years'
                )],
            )
        ],
        style={'width': '100%',              
               'display': 'inline-block'}),
    dcc.Graph(id='graph')
])


@app2.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('Genres', 'value'), dash.dependencies.Input('Years', 'value')])
def update_graph(Genres, Years):
    Years = range(Years[0], Years[1])    
    
    heatmap_matrix = np.zeros((len(Genres), len(Years)))
    
    for i in range(len(Genres)):
        for j in range(len(Years)):
            aux = movies_genres.loc[((movies_genres['Genre 1'] == Genres[i]) | (movies_genres['Genre 2'] == Genres[i]) | (movies_genres['Genre 3'] == Genres[i])) & (movies_genres['Released_Year'] == Years[j]), 'Gross'].mean()
            if math.isnan(aux):
                heatmap_matrix[i][j] = 0
            else:
                heatmap_matrix[i][j] = aux
    heatmap_df = pd.DataFrame(heatmap_matrix, columns = Years, index = Genres)
    
    return px.imshow(heatmap_df, text_auto=True, aspect="auto",
                labels=dict(x="Anos", y="Gênero", color="Receita"), title="Receita média dos filmes por ano e por gênero"
               )


if __name__ == '__main__':
    app2.run_server(debug=True, port="8052")